In [1]:
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from PIL import Image
import glob as glob
import os
import random
import math

ONB_EPSILON = 0.01

In [2]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm

In [3]:
def initFromU(u):
    n = np.array([0.0, 1.0, 0.0])
    m = np.array([0.0, 0.0, 1.0])

    U = normalize(u)
    V = np.cross(U, n);
    if (np.linalg.norm(V) < ONB_EPSILON):
        V = np.cross(U, m)
    W = np.cross(U,V)
    return U, V, W

In [4]:
def getColors(n):
    ret = []
    r = int(random.random() * 256)
    g = int(random.random() * 256)
    b = int(random.random() * 256)
    step = 256 / n
    for i in range(n):
        r += step
        g += step
        b += step
        r = int(r) % 256
        g = int(g) % 256
        b = int(b) % 256
        ret.append((r,g,b)) 
    return np.asarray(ret)

In [5]:
def getPhiTheta(filename):
    file_name = os.path.basename(filename)
    name, ext = os.path.splitext(file_name)
    phi, theta = name.split('_')[-3], name.split('_')[-1]
    return int(phi), int(theta)

In [58]:
initFromU([0,1,0])

(array([0., 1., 0.]), array([1., 0., 0.]), array([ 0.,  0., -1.]))

In [6]:
def localToWorld(point, normal):
    transformation_matrix = np.zeros((4,4))
    
    # Initialize fourth column of viewing matrix
    transformation_matrix[:, 3] = np.append(point, 1)
    
    #Initialize first three columns of viewing matrix
    
    right, newup, dir = initFromU(normal)
    
    transformation_matrix[:, 0] = np.append(right, 0)
    transformation_matrix[:, 1] = np.append(newup, 0) # check with (-) sign
    transformation_matrix[:, 2] = np.append(dir, 0)
    
    return transformation_matrix, np.linalg.inv(transformation_matrix)

In [7]:
def generateShapeCodedImages(filename, delim = ', ', sizex = 256, sizey =256):
    content = pd.read_csv(filename, delimiter=delim, engine= 'python').to_numpy()
    total_shapes = len(np.unique(content[:,3]))
    print (str(total_shapes) + ' shapes detected for this image')
    shape_indices = np.unique(content[:,3])
    colors = np.random.rand(shape_indices.shape[0],3)*255
    m_out = np.zeros((sizex, sizey, total_shapes))
    matrix_out = np.zeros((sizex,sizey,3))

    for i in tqdm(range(len(content))):
        c = content[i]
        x, y, sid, pid, px, py, pz, nx, ny, nz = c
        m_out[int(x),int(y), np.where(shape_indices==pid)[0][0] ] += 1
        #matrix_out[int(y),int(x)] = colors[np.where(shape_indices==sid)[0][0]]
    
    for x in tqdm(range(sizex)):
        for y in range(sizey):
            if m_out[x][y].max() > 0.9:
                max_shape = np.argmax(m_out[x][y])
                matrix_out[int(y),int(x)] = colors[max_shape]
            
    im = Image.fromarray(np.uint8(matrix_out))
    return im

In [8]:
## Assumes left hand coordinate system. Phi is from thr up/y axis, clockwise, theta is x/right axis anticlock wise.

def getPhi(x, y, z):
    value = math.degrees(math.atan2(math.sqrt(x**2+z**2), y))
    if value >= 0:
        return value 
    return (value+360)%360
def getTheta(x, y, z):
    value = math.degrees(math.atan2(z, x)) # x is the standard
    if value >= 0:
        return value 
    return (value+360)%360
    
def getX(radius, theta, phi):
    return radius * math.cos(theta*0.01745) * math.sin(phi*0.01745);

def getZ(radius, theta, phi):
    return radius * math.sin(theta*0.01745) * math.sin(phi*0.01745);

def getY(radius, theta, phi):
    return radius * math.cos(phi*0.01745);

In [9]:
def phiIndexGenerator(phi_n, phi_c):
    #return abs(phi_n-phi_c)
    lower = min(phi_n, phi_c)
    higher = max(phi_n, phi_c)
    basic_diff = abs(phi_n-phi_c)
    alter_diff = abs(lower+360-higher)
    return min(basic_diff, alter_diff)

def thetaIndexGenerator(theta_n, theta_c):
    lower = min(theta_n, theta_c)
    higher = max(theta_n, theta_c)
    basic_diff = abs(theta_n-theta_c)
    alter_diff = abs(lower+360-higher)
    return min(basic_diff, alter_diff)

In [10]:
PATH = '/home/farhan/Downloads/bunny/'
im = generateShapeCodedImages(PATH + 'data/data_phi_46_theta_231.txt')
display(im)

FileNotFoundError: [Errno 2] No such file or directory: '/home/farhan/Downloads/bunny/data/data_phi_46_theta_231.txt'

## Writing reflectance values to files

In [11]:
PATH = '/home/farhan/Downloads/bunny/'
image_files = sorted(glob.glob(PATH + 'images/*.png'))
log_files = sorted(glob.glob(PATH + 'data/*.txt'))

In [12]:
image_files[0], log_files[0]

('/home/farhan/Downloads/bunny/images/render_phi_20_theta_0.png',
 '/home/farhan/Downloads/bunny/data/data_phi_20_theta_0.txt')

In [ ]:
for l in tqdm(range(len(log_files))):
    assert os.path.splitext(os.path.basename(image_files[l]))[0][7:] == os.path.splitext(os.path.basename(log_files[l]))[0][5:]
        
    loaded_log = np.loadtxt(log_files[l], delimiter = ', ')
    #loaded_log = np.expand_dims(loaded_log, axis = 0)
    phi_c, theta_c = getPhiTheta(log_files[l])
    
    image_file_name = PATH+'images/render' + os.path.splitext(os.path.basename(log_files[l]))[0][4:] + '.png'
    loaded_image = np.asarray((Image.open(image_file_name)).convert('RGB'))
    
    for r in range(len(loaded_log)):
        row = loaded_log[r]
        x, y, sid, pid, px, py, pz, nx, ny, nz, bull1, bull2, bull3 = row
        
        #phi_n, theta_n = getPhi(nx, ny, nz), getTheta(nx, ny, nz)
        #phi, theta = phiIndexGenerator(phi_n, phi_c), thetaIndexGenerator(theta_n, theta_c)
                
        r, g, b = loaded_image[int(y)][int(x)]
        if not (os.path.isdir(PATH + 'bunny_reflectance_maps/'  + str(int(pid)))):
            os.mkdir(PATH + 'bunny_reflectance_maps/'  + str(int(pid)))
        with open(PATH + 'bunny_reflectance_maps/' + str(int(pid)) + '/' + str(int(pid)) + '_' + str(int(sid)) + '.txt', 'a+') as f:
            #temp = np.array([phi_c, theta_c, px, py, pz, r, g, b])
            f.write(str(phi_c) + ', ' + str(theta_c) + ', ' + str(px) + ', ' + str(py) + ', ' + str(pz) + ', ' + str(nx) + ', ' + str(ny) + ', ' + str(nz) + ', '  + str(r) + ', ' + str(g) + ', ' + str(b) + '\n')
            #np.savetxt(f, temp, delimiter = ', ', newline = '\n')

## From reflectance map files to map images

In [122]:
all_map_dirs = os.listdir(PATH+'reflectance_maps')
#map_files = sorted(glob.glob(PATH + 'reflectance_maps/57/*.txt'), key=os.path.getmtime)

In [72]:
getTheta(0,0,-10)

270.0

In [124]:
for od in tqdm(range(0,len(all_map_dirs))):
    one_dir = all_map_dirs[od]
    map_file = sorted(glob.glob(PATH + 'reflectance_maps/' + one_dir + '/*.txt'), key=os.path.getsize)[-1]
    
    map_image = np.zeros((180, 360, 4), dtype = 'int')
    one_map_file = np.loadtxt(map_file, delimiter = ', ', dtype = 'float16')
    if (len(one_map_file.shape) < 2):
        one_map_file = np.expand_dims(one_map_file, axis=0)
    for i in tqdm(range(len(one_map_file)-1,0,-1)):
        assert type(i)==int
        phi, theta, px, py, pz, nx, ny, nz, r, g, b = one_map_file[i]
        cam_x, cam_y, cam_z = getX(7, theta, phi), getY(7, theta, phi), getZ(7, theta, phi)
        l2w, w2l = localToWorld([px, py, pz], [nx, ny, nz])
        cam_local = np.matmul(w2l, [cam_x, cam_y, cam_z, 1])
        cam_x, cam_y, cam_z = [cam_local[0], cam_local[1], cam_local[2]]/cam_local[3]
        cam_phi, cam_theta = getPhi(cam_x, cam_y, cam_z), getTheta(cam_x, cam_y, cam_z)
#         if(map_image[int(phi), int(theta)].all() <= 0.0001):
#         map_image[int(phi/5), int(theta/5)] = r, g, b
#         else:
        if(r <= 0.0001 and g <= 0.0001 and b <= 0.0001):
            continue
        map_image[int(cam_phi), int(cam_theta), :3] +=  np.array([r, g, b], dtype='int')
        map_image[int(cam_phi), int(cam_theta), 3] += 1
    map_image = map_image[:,:,:3]/np.expand_dims(map_image[:,:,3], axis=2)
    reflectance_image = Image.fromarray(np.uint8(map_image))
    reflectance_image.save(PATH + 'maps/map_' + one_dir + '.png')
        

/home/farhan/anaconda3/envs/kaolin/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide


In [104]:
map_image = map_image[:,:,:3]/np.expand_dims(map_image[:,:,3], axis=2)
reflectance_image = Image.fromarray(np.uint8(map_image))
reflectance_image.save(PATH + 'maps/map_' + one_dir + '.png')

/home/farhan/anaconda3/envs/kaolin/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [72]:
all_map_dirs[0]+'wasee'

'30wasee'

In [61]:
for m in tqdm(range(len(map_files))):
    map_image = np.zeros((18, 72, 3), dtype = 'int')
    one_map_file = np.loadtxt(map_files[m], delimiter = ', ', dtype = 'float16')
    if (len(one_map_file.shape) < 2):
        one_map_file = np.expand_dims(one_map_file, axis=0)
    for i in tqdm(range(len(one_map_file))):
        assert type(i)==int
        phi, theta, px, py, pz, r, g, b = one_map_file[i]
        #if(r <= 0.0001 and g <= 0.0001 and b <= 0.0001):
        #    continue
#         if(map_image[int(phi), int(theta)].all() <= 0.0001):
#         map_image[int(phi/5), int(theta/5)] = r, g, b
#         else:
        map_image[int(phi/5), int(theta/5)] = [r, g, b]#(map_image[int(phi/5), int(theta/5)] + [r, g, b])/2
        
    reflectance_image = Image.fromarray(np.uint8(map_image))
    reflectance_image.save(PATH + 'maps/map_' + str(m+1) + '.png')
    

## Theta-Phi range check

In [34]:
content = np.loadtxt('/home/farhan/Masters_Thesis/pbrt_experiments/cube_obj/reflectance_maps/1.rtx', delimiter = ', ', dtype = 'float16')
for i in tqdm(range(len(content))):
        phi, theta, r, g, b = content[i]
        print(str(phi/5) + ' ' + str(theta/5))

0.2 68.2
3.2 0.2
3.2 0.2
3.2 0.2
3.2 0.2
3.2 0.2
3.2 0.2
3.2 0.2
3.2 0.2
3.2 0.2
3.2 0.2
3.2 0.2
3.2 0.2
3.2 0.2
3.2 0.2
3.2 0.2
3.2 0.2
3.2 0.2
3.2 0.2
3.2 0.2
3.2 0.2
3.2 0.2



In [33]:
one_map_file.shape

(39777002, 5)

In [40]:
one_map_file[:,0].max(), one_map_file[:,1].max()

(89.0, 124.0)

In [44]:
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_name('efficientnet-b4')


In [ ]:
11, 351, 0.038235, 0.952021, -0.303636, 0.038235, 0.952021, -0.303636,

In [61]:
cam_x, cam_y, cam_z = getX(2, 1, 126), getY(2, 1, 126), getZ(2, 1, 126)

In [62]:
l2w, w2l = localToWorld([0.1648,   0.928 ,  -0.334],[0.1648, 0.928, -0.334])

In [63]:
cam_space = np.matmul(w2l, [cam_x, cam_y, cam_z, 1])

In [64]:
cam_space

array([-1.83303899,  3.92985949,  2.89595509,  1.        ])

In [65]:
getPhi(cam_space[0], cam_space[1], cam_space[2]), getTheta(cam_space[0], cam_space[1], cam_space[2])

(41.09250333802548, 122.33238974221699)

In [60]:
one_map_file[1325432]

array([  1.    , 126.    ,   0.1648,   0.928 ,  -0.334 ,   0.1648,
         0.928 ,  -0.334 ,  53.    ,  27.    ,  27.    ], dtype=float16)